Thanks pivotmesh
Weng H, Wang Y, Zhang T, et al. Pivotmesh: Generic 3d mesh generation via pivot vertices guidance[J]. arXiv preprint arXiv:2405.16890, 2024.

In [ ]:
!pip install trimesh
!pip install scipy

In [ ]:
import os
import trimesh
import random
import networkx as nx
import numpy as np
from six.moves import range
from scipy.spatial.transform import Rotation

def save_mesh_to_obj(pts, polygons, save_path, reorder_axes=True, scale_factor=1):
    if reorder_axes:
        pts = pts[:, [2, 1, 0]]
    pts *= scale_factor
    if polygons is not None:
        index_offset = 1 if min(min(polygons)) == 0 else 0
    with open(save_path, "w") as file:
        for p in pts:
            file.write("v {} {} {}\n".format(p[0], p[1], p[2]))
        for poly in polygons:
            face_line = "f"
            for idx in poly:
                face_line += " {}".format(idx + index_offset)
            face_line += "\n"
            file.write(face_line)

def quantize_points(pts, bits=6):
    clamp = 0.5
    min_val = -clamp
    max_val = clamp
    levels = 2 ** bits
    quantized = (pts - min_val) * levels / (max_val - min_val)
    return quantized.astype("int32")

def extract_face_cycles(polygon):
    graph = nx.Graph()
    for i in range(len(polygon) - 1):
        graph.add_edge(polygon[i], polygon[i + 1])
    graph.add_edge(polygon[-1], polygon[0])
    return list(nx.cycle_basis(graph))

def serialize_faces(polygons):
    if not polygons:
        return np.array([0])
    padded = [face + [-1] for face in polygons[:-1]] + [polygons[-1] + [-2]]
    return (np.array([v for face in padded for v in face]) + 2)

def recenter_points(pts):
    lower = pts.min(axis=0)
    upper = pts.max(axis=0)
    center = 0.5 * (lower + upper)
    return pts - center

def scale_to_unit_diagonal(pts):
    lower = pts.min(axis=0)
    upper = pts.max(axis=0)
    diag = np.linalg.norm(upper - lower)
    return pts / (diag + 1e-6)

def deduplicate_and_quantize(pts, polys, triangles=None, bits=6):
    pts = quantize_points(pts, bits)
    pts, inverse_map = np.unique(pts, axis=0, return_inverse=True)
    reorder_indices = np.lexsort(pts[:, ::-1].T)
    pts = pts[reorder_indices]
    polys = [np.argsort(reorder_indices)[inverse_map[f]] for f in polys]
    if triangles is not None:
        triangles = np.array([np.argsort(reorder_indices)[inverse_map[t]] for t in triangles])
    # 修改循环部分，保持原始顶点顺序
    cleaned = []
    for face in polys:
        if len(face) < 3:
            continue  # 跳过无效面
        # 直接使用原始面，确保顶点顺序不变
        cleaned.append(face.tolist())
    polys = cleaned
    if triangles is not None:
        triangles = np.array([t for t in triangles if len(set(t)) == len(t)])
    polys.sort(key=lambda f: tuple(sorted(f)))
    if triangles is not None:
        triangles = triangles.tolist()
        triangles.sort(key=lambda f: tuple(sorted(f)))
        triangles = np.array(triangles)
    used = np.isin(np.arange(pts.shape[0]), np.hstack(polys))
    pts = pts[used]
    idx_map = np.arange(len(used)) - np.cumsum(~used)
    polys = [idx_map[face].tolist() for face in polys]
    if triangles is not None:
        triangles = np.array([idx_map[t].tolist() for t in triangles])
    return pts, polys, triangles

#
def prepare_mesh_data(pts, polys, bits=6, flatten=False, apply_aug=True, aug_params=None):
    pts = recenter_points(pts)
    pts = scale_to_unit_cube(pts)  # 替换为新的归一化函数
    if apply_aug:
        pts = apply_random_transform(pts, **aug_params)
    pts, polys, _ = deduplicate_and_quantize(pts, polys, bits=bits)
    if flatten:
        polys = serialize_faces(polys)
    return {"vertices": pts, "faces": polys}
def scale_to_unit_cube(pts):
    if pts.size == 0:
        return pts
    lower = pts.min(axis=0)
    upper = pts.max(axis=0)
    center = (lower + upper) / 2
    pts_centered = pts - center
    max_side = (upper - lower).max()
    if max_side == 0:
        return pts_centered
    return pts_centered / max_side

def read_and_process_obj(mesh_path, bits=6, apply_aug=False, aug_params=None):
    mesh = trimesh.load(mesh_path, force='mesh', process=False)
    return prepare_mesh_data(mesh.vertices, mesh.faces, bits, apply_aug=apply_aug, aug_params=aug_params)

#
def apply_random_transform(pts, scale_range_min=0.95, scale_range_max=1.05, rot_angle=0.0, jitter_std=0.0):
    for axis in range(3):
        factor = random.uniform(scale_range_min, scale_range_max)
        pts[:, axis] *= factor
    if rot_angle != 0.0:
        # 生成随机单位向量
        rand_axis = np.random.randn(3)
        rand_axis /= np.linalg.norm(rand_axis) + 1e-8  # 避免除以零
        radians = np.radians(rot_angle)
        rot_matrix = Rotation.from_rotvec(rand_axis * radians)
        pts = rot_matrix.apply(pts)
    if jitter_std != 0.0:
        jitter = np.random.normal(scale=jitter_std, size=pts.shape)
        pts += jitter
    return pts

#
def batch_process_objs(src_dir, dst_dir, reorder=True):
    os.makedirs(dst_dir, exist_ok=True)  # 使用exist_ok避免竞态条件
    for fname in os.listdir(src_dir):
        if not fname.lower().endswith('.obj'):
            continue
        src_path = os.path.join(src_dir, fname)
        dst_path = os.path.join(dst_dir, fname)
        if os.path.exists(dst_path):
            print(f"跳过已存在文件: {fname}")
            continue
        try:
            data = read_and_process_obj(src_path, bits=6, apply_aug=False)  # 调整bits为6
            save_mesh_to_obj(data["vertices"], data["faces"], dst_path, reorder_axes=reorder)
            # print(f"成功处理: {fname}")
        except Exception as e:
            print(f"处理 {fname} 失败，错误: {e}")
            # 可选：将错误文件移动到特定目录或记录日志

input_path = ''
output_path = ''
reorder_flag = False
batch_process_objs(input_path, output_path, reorder_flag)

In [ ]:
def save_mesh_to_obj(pts, polygons, save_path, reorder_axes=True, scale_factor=1):
    if reorder_axes:
        pts = pts[:, [2, 1, 0]]
    pts *= scale_factor
    if polygons is not None:
        index_offset = 1 if min(min(polygons)) == 0 else 0
    with open(save_path, "w") as file:
        for p in pts:
            file.write("v {} {} {}\n".format(p[0], p[1], p[2]))
        for poly in polygons:
            face_line = "f"
            for idx in poly:
                face_line += " {}".format(idx + index_offset)
            face_line += "\n"
            file.write(face_line)

def quantize_points(pts, bits=6):
    clamp = 0.5
    min_val = -clamp
    max_val = clamp
    levels = 2 ** bits
    quantized = (pts - min_val) * levels / (max_val - min_val)
    return quantized.astype("int32")

def extract_face_cycles(polygon):
    graph = nx.Graph()
    for i in range(len(polygon) - 1):
        graph.add_edge(polygon[i], polygon[i + 1])
    graph.add_edge(polygon[-1], polygon[0])
    return list(nx.cycle_basis(graph))

def serialize_faces(polygons):
    if not polygons:
        return np.array([0])
    padded = [face + [-1] for face in polygons[:-1]] + [polygons[-1] + [-2]]
    return (np.array([v for face in padded for v in face]) + 2)

def recenter_points(pts):
    lower = pts.min(axis=0)
    upper = pts.max(axis=0)
    center = 0.5 * (lower + upper)
    return pts - center

def scale_to_unit_diagonal(pts):
    lower = pts.min(axis=0)
    upper = pts.max(axis=0)
    diag = np.linalg.norm(upper - lower)
    return pts / (diag + 1e-6)

def deduplicate_and_quantize(pts, polys, triangles=None, bits=6):
    pts = quantize_points(pts, bits)
    pts, inverse_map = np.unique(pts, axis=0, return_inverse=True)
    reorder_indices = np.lexsort(pts[:, ::-1].T)
    pts = pts[reorder_indices]
    polys = [np.argsort(reorder_indices)[inverse_map[f]] for f in polys]
    if triangles is not None:
        triangles = np.array([np.argsort(reorder_indices)[inverse_map[t]] for t in triangles])
    # 修改循环部分，保持原始顶点顺序
    cleaned = []
    for face in polys:
        if len(face) < 3:
            continue  # 跳过无效面
        # 直接使用原始面，确保顶点顺序不变
        cleaned.append(face.tolist())
    polys = cleaned
    if triangles is not None:
        triangles = np.array([t for t in triangles if len(set(t)) == len(t)])
    polys.sort(key=lambda f: tuple(sorted(f)))
    if triangles is not None:
        triangles = triangles.tolist()
        triangles.sort(key=lambda f: tuple(sorted(f)))
        triangles = np.array(triangles)
    used = np.isin(np.arange(pts.shape[0]), np.hstack(polys))
    pts = pts[used]
    idx_map = np.arange(len(used)) - np.cumsum(~used)
    polys = [idx_map[face].tolist() for face in polys]
    if triangles is not None:
        triangles = np.array([idx_map[t].tolist() for t in triangles])
    return pts, polys, triangles

#
def prepare_mesh_data(pts, polys, bits=6, flatten=False, apply_aug=True, aug_params=None):
    pts = recenter_points(pts)
    pts = scale_to_unit_cube(pts)  # 替换为新的归一化函数
    if apply_aug:
        pts = apply_random_transform(pts, **aug_params)
    pts, polys, _ = deduplicate_and_quantize(pts, polys, bits=bits)
    if flatten:
        polys = serialize_faces(polys)
    return {"vertices": pts, "faces": polys}
def scale_to_unit_cube(pts):
    if pts.size == 0:
        return pts
    lower = pts.min(axis=0)
    upper = pts.max(axis=0)
    center = (lower + upper) / 2
    pts_centered = pts - center
    max_side = (upper - lower).max()
    if max_side == 0:
        return pts_centered
    return pts_centered / max_side

def read_and_process_obj(mesh_path, bits=6, apply_aug=False, aug_params=None):
    mesh = trimesh.load(mesh_path, force='mesh', process=False)
    return prepare_mesh_data(mesh.vertices, mesh.faces, bits, apply_aug=apply_aug, aug_params=aug_params)

#
def apply_random_transform(pts, scale_range_min=0.95, scale_range_max=1.05, rot_angle=0.0, jitter_std=0.0):
    for axis in range(3):
        factor = random.uniform(scale_range_min, scale_range_max)
        pts[:, axis] *= factor
    if rot_angle != 0.0:
        # 生成随机单位向量
        rand_axis = np.random.randn(3)
        rand_axis /= np.linalg.norm(rand_axis) + 1e-8  # 避免除以零
        radians = np.radians(rot_angle)
        rot_matrix = Rotation.from_rotvec(rand_axis * radians)
        pts = rot_matrix.apply(pts)
    if jitter_std != 0.0:
        jitter = np.random.normal(scale=jitter_std, size=pts.shape)
        pts += jitter
    return pts

#
def batch_process_objs(src_dir, dst_dir, reorder=True):
    os.makedirs(dst_dir, exist_ok=True)  # 使用exist_ok避免竞态条件
    for fname in os.listdir(src_dir):
        if not fname.lower().endswith('.obj'):
            continue
        src_path = os.path.join(src_dir, fname)
        dst_path = os.path.join(dst_dir, fname)
        if os.path.exists(dst_path):
            print(f"跳过已存在文件: {fname}")
            continue
        try:
            data = read_and_process_obj(src_path, bits=6, apply_aug=False)  
            save_mesh_to_obj(data["vertices"], data["faces"], dst_path, reorder_axes=reorder)
            print(f"成功处理: {fname}")
        except Exception as e:
            print(f"处理 {fname} 失败，错误: {e}")
            # 可选：将错误文件移动到特定目录或记录日志

input_path = './data_test/new1'
output_path = './data_test/4'
reorder_flag = True
batch_process_objs(input_path, output_path, reorder_flag)